# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv = "../python-api-challenge/output/cities.csv"

city_weather = pd.read_csv(csv)

city_weather.head()
#ideal_climate_df[0:10].itterrows()


,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.6500,115.3333,51.80,82,53,4.92,AU,2022-08-10
1,1,roald,62.5833,6.1333,66.00,94,75,27.63,NO,2022-08-10
2,2,coos bay,43.3665,-124.2179,79.32,56,0,9.22,US,2022-08-10
3,3,cherskiy,68.7500,161.3000,60.30,65,0,2.86,RU,2022-08-10
4,4,atuona,-9.8000,-139.0333,77.32,76,13,22.08,PF,2022-08-10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)
locations = city_weather[['Lat', 'Lon']]
weights = city_weather['Humidity']

layout = {
    'width':'100%',
    'height':'400px',
    'border':'1px sold black',
    'padding': '1px'
}
fig = gmaps.figure(layout=layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=100, point_radius= 1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(border='1px sold black', height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
ideal_weather = city_weather.loc[
    (city_weather['Max Temp']> 70) &
    (city_weather['Max Temp']<90) &
    (city_weather['Humidity']<70) &
    (city_weather['Cloudiness']<50) &
    (city_weather['Wind Speed']<20) &
    (city_weather['Country'] != 'RU')
]
ideal_weather.reset_index()
ideal_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,coos bay,43.3665,-124.2179,79.32,56,0,9.22,US,2022-08-10
28,28,west fargo,46.8750,-96.9004,80.02,39,0,5.75,US,2022-08-10
29,29,avarua,-21.2078,-159.7750,77.05,69,20,14.97,CK,2022-08-10
36,36,wahpeton,46.2652,-96.6059,83.12,34,0,5.75,US,2022-08-10
43,43,marathon,44.9000,-89.7668,85.35,59,0,9.46,US,2022-08-10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = ideal_weather[['City', 'Lat', 'Lon', 'Country']].copy()
hotel_df['Hotel Name'] = np.nan
hotel_df = hotel_df.rename(
    columns={
        'Lon':'Lng'
    }
)
hotel_df.head()

,City,Lat,Lng,Country,Hotel Name
2,2,43.3665,-124.2179,US,NaN
28,28,46.8750,-96.9004,US,NaN
29,29,-21.2078,-159.7750,CK,NaN
36,36,46.2652,-96.6059,US,NaN
43,43,44.9000,-89.7668,US,NaN


In [43]:
ideal_weather.head()

,City,City.1,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,coos bay,43.3665,-124.2179,79.32,56,0,9.22,US,2022-08-10
28,28,west fargo,46.8750,-96.9004,80.02,39,0,5.75,US,2022-08-10
29,29,avarua,-21.2078,-159.7750,77.05,69,20,14.97,CK,2022-08-10
36,36,wahpeton,46.2652,-96.6059,83.12,34,0,5.75,US,2022-08-10
43,43,marathon,44.9000,-89.7668,85.35,59,0,9.46,US,2022-08-10


In [45]:
# I need a place to sleep after all of my fun
target_search = 'hotel'
target_radius = 5000
target_type = 'lodging'

params = {
    'keyword': target_search,
    'radius': target_radius,
    'type': target_type,
    'key': g_key
}
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df[0:10].iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat}, {lng}"

    hotel_data = requests.get(url, params).json()

# Save the first hotel name for each city
    try:
        hotel_name = hotel_data['results'][0]['name']
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        print(f"City '{row['City']}' found: The nearest hotel is {hotel_name} ")
        print("----------------------")
    except:
        print(f"Couldn't find any information for city '{row['City']}' . . . skipping.") 
        print("----------------------")

City '2' found: The nearest hotel is Best Western Holiday Hotel 
----------------------
City '28' found: The nearest hotel is Holiday Inn Express Fargo SW - I-94 Medical Center, an IHG Hotel 
----------------------
City '29' found: The nearest hotel is The Islander Hotel 
----------------------
City '36' found: The nearest hotel is AmericInn by Wyndham Wahpeton 
----------------------
Couldn't find any information for city '43' . . . skipping.
----------------------
Couldn't find any information for city '57' . . . skipping.
----------------------
City '61' found: The nearest hotel is Norwalk Inn 
----------------------
City '70' found: The nearest hotel is Lisboa Palace Hotel 
----------------------
City '71' found: The nearest hotel is Best Western Thompson Hotel & Suites 
----------------------
City '76' found: The nearest hotel is Shelburne motel 
----------------------


In [47]:
hotel_df.dropna(how="any", inplace=True)
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Lat,Lng,Country,Hotel Name
0,2,43.3665,-124.2179,US,Best Western Holiday Hotel
1,28,46.8750,-96.9004,US,Holiday Inn Express Fargo SW - I-94 Medical Ce...
2,29,-21.2078,-159.7750,CK,The Islander Hotel
3,36,46.2652,-96.6059,US,AmericInn by Wyndham Wahpeton
4,61,41.1415,-73.3579,US,Norwalk Inn
5,70,-9.8756,-56.0861,BR,Lisboa Palace Hotel
6,71,55.7435,-97.8558,CA,Best Western Thompson Hotel & Suites
7,76,44.0787,-80.2041,CA,Shelburne motel


In [48]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [49]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px sold black', height='400px', padding='1px', width='100%'))